# Problem Set 3 CS174
---
The following problem set will focus on the lessons on Bag of Words and TFIDF. Answer the following guide questions with your groups by writing functions or code in cells that accomplish the requirements.

1. Take your favorite Spotify or Apple Music playlist, and use their APIs to programmatically extract the first 20 song titles. Build a dictionary that has the song index as the key, and the title as the value. Please make sure that the playlist is **NOT** an instrumental playlist. **3 pts.**

2. Use the Genius API to programmatically download the lyrics of these songs and store them in a dictionary with the index as the key, and the lyrics as the value. **3 pts.**

3. Build a 20x20 matrix containing the cosine similarities of the songs to each other. Use Bag of Words to determine these similarities. **4 pts.**


4. Build a 20x20 matrix containing the cosine similarities of the songs to each other. Use TFIDF scores to determine these similarities. **5 pts.**

5. Compare the similarities using BoW and TFIDF. Analyze the results and discuss any findings that interest you. You can use heatmaps or other plots to present your analysis. **3 pts**

Guide Questions:
- Can this be used to determine playlist quality?
- What does this say about the homogeneity of the playlist themes?
- What does this say about how songs (in that playlist genre) are written?

6. Identify the top 5 most important words for each song using TFIDF. Discuss the relationship of these words to their respective songs and analyze if they can be used as passable summaries of the songs. **2 pts** 

Make sure to remove stopwords. No need to lemmatize or stem, but is very welcome.
       
### Bonus:
Do a TFIDF analysis for a single artist's top 5 songs. Identify the top 10 most important words and discuss an analysis of these. **5 pts.**

Deadline **March 17 11:59PM**. Submission link to be posted in Moodle. 
Submit a .zip file containing the notebook and a 'data/' directory containing the songs with name <SURNAME>_<ID NUMBER>.ipynb.
Make sure to remove or obfuscate any API keys you include in the final submission.
    
Sample: **"BAUTISTA_110464.zip"**

In [4]:
# number 1
import spotipy
import spotipy.oauth2  as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import nltk
import numpy as np
import random
import string, re
import seaborn as sns
import matplotlib.pyplot as plt
import heapq
%pprint


auth_manager = SpotifyClientCredentials(client_id='93c0ab7ed3d74a828649b0c2f870f219', client_secret='3cca9de3137c49cc969ea411d5f6507e')

def getTrackIDs(user, playlist_id):
    track_ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
            track = item['track']
            track_ids.append(track['id'])
    return track_ids

sp = spotipy.Spotify(auth_manager=auth_manager)
track_ids = getTrackIDs('spotify', '37i9dQZF1DWSY3qrD2Jh0k')

print("Total Songs in Album:", len(track_ids))
print("The Track IDs in the Album: ", track_ids)

def getTrackFeatures(id):
    track_info = sp.track(id)
    
    name = track_info['name']
    
    artist = track_info['album']['artists'][0]['name']

    track_data = [name,artist]
    
    return track_data

dictionary= []
artist=[]

for i in range(len(track_ids)):
    time.sleep(.3)
    track_data = getTrackFeatures(track_ids[i])[0]
    track_artist = getTrackFeatures(track_ids[i])[1]
    artist.append(track_artist)
    dictionary.append(track_data)
    
road_trip = pd.DataFrame(dictionary, columns = ['Name'])
road_trip_artists = pd.DataFrame(artist, columns = ['artist'])
road_trip.head(20)
road_trip_artists.head(20)
frames=[road_trip,road_trip_artists]
result=pd.concat(frames,axis=1)
result.index.name='key'
result=result.head(20)
result

Pretty printing has been turned ON
Total Songs in Album: 56
The Track IDs in the Album:  ['2P91MQbaiQOfbiz9VqhqKQ', '1yDiru08Q6omDOGkZMPnei', '0Eqg0CQ7bK3RQIMPw1A7pl', '6mQLN3zRtAp6ovjusyYKrV', '4zRZAmBQP8vhNPf9i9opXt', '6aYwwMcANnpuUm4zLsCzy3', '6VrLYoQKdhu1Jruei06t65', '3WUEs51GpcvlgU7lehLgLh', '4Pbg79cTBu4vgSphoyNq3j', '3id2EVGKc3eFAMn9nFnB0r', '1X4l4i472kW5ofFP8Xo0x0', '5uCax9HTNlzGybIStD3vDh', '2hnxrRNzF74mdDzpQZQukQ', '69vToJ9BMbbLlFZo7k7A7B', '4tNXntkAzQ5A2dfYRYGIIQ', '06cUybZMETs4i3yQO1f7p3', '4hQ6UGyWQIGJmHSo0J88JW', '5eFzCfsj3d0yWumdEldZwL', '3EPXxR3ImUwfayaurPi3cm', '7ArrTJ1LwcGAlEO65LUQ7i', '0prNGof3XqfTvNDxHonvdK', '11cDEev79fPKi6uJCjUhu8', '79jX8RM5CgPqDKdGEKNW9K', '4OBZT9EnhYIV17t4pGw7ig', '5NXdUJ3Z2jhlp2u1cj6f7m', '5p7ujcrUXASCNwRaWNHR1C', '05CwHjIk71RXVU40boRMnR', '3RauEVgRgj1IuWdJ9fDs70', '7GbAp0HKPQW7WnFJAzMoRk', '5PYQUBXc7NYeI1obMKSJK0', '4keoy2fqgwGnbWlm3ZVZFa', '5cXAhazDamL2rcIxThZWOt', '4NBTZtAt1F13VvlSKe6KTl', '4y3OI86AEP6PQoDE6olYhO', '2BgD4nRyx9EZ5o8YEnjRSV', 

,Name,artist
key,,
0,I Like Me Better,Lauv
1,Maybe The Night,Ben&Ben
2,Malibu Nights,LANY
3,Rewrite The Stars (with James Arthur & Anne-Ma...,James Arthur
4,8 Letters,Why Don't We
5,tomorrow tonight,Loote
6,Somebody To You,The Vamps
7,Kathang Isip,Ben&Ben
8,2002,Anne-Marie


In [41]:
# number 2
import lyricsgenius
d={}
genius_access_token='_7Gu-NHaJlETfuOEBUU8h8O6yPZBgGgzpbyCE_P9ryheBAp9t3aYflgPuZXo_LrR'
genius = lyricsgenius.Genius(genius_access_token)
num =  len(result)
s=result['artist']
r=result['Name']
i=0
while i<num:
    
    artist = genius.search_artist(s[i], max_songs=0, sort="title")
    song=artist.song(r[i])
    lyrics=song.lyrics
    d[i]=lyrics.replace('\n',' ')
    
    i+=1

Searching for songs by Lauv...

Done. Found 0 songs.
Searching for "I Like Me Better" by Lauv...
Done.
Searching for songs by Ben&Ben...

Done. Found 0 songs.
Searching for "Maybe The Night" by Ben&Ben...
Done.
Searching for songs by LANY...

Done. Found 0 songs.
Searching for "Malibu Nights" by LANY...
Done.
Searching for songs by James Arthur...

Done. Found 0 songs.
Searching for "Rewrite The Stars (with James Arthur & Anne-Marie)" by James Arthur...
Done.
Searching for songs by Why Don't We...

Changing artist name to 'Why Don’t We'
Done. Found 0 songs.
Searching for "8 Letters" by Why Don’t We...
Done.
Searching for songs by Loote...

Done. Found 0 songs.
Searching for "tomorrow tonight" by Loote...
Done.
Searching for songs by The Vamps...

Done. Found 0 songs.
Searching for "Somebody To You" by The Vamps...
Done.
Searching for songs by Ben&Ben...

Done. Found 0 songs.
Searching for "Kathang Isip" by Ben&Ben...
Done.
Searching for songs by Anne-Marie...

Done. Found 0 songs.
Sear

In [42]:
#number 3 is everything onwards
temp=0
songs=[]
while temp<len(d):
    songs.append(d[temp])
    
    temp+=1


In [73]:
temp=0
freq_words_list=[]
while temp<len(d):    
    
    stopwords = set(line.strip() for line in open('data/stop.txt'))
    words= [[re.sub('[^A-Za-z0-9]+','',word)for word in sentence.split(" ")]for sentence in songs]
    
    wordfreq = {}
    for sentence in words:
        for word in sentence:
            word = word.replace(".","")
            word = word.replace(",","")
            word = word.replace(":","")
            word = word.replace("\"","")
            word = word.replace("!","")
            word = word.lower()
            if word not in wordfreq.keys() and word not in stopwords:
                wordfreq[word]=1
            else:
                wordfreq[word]+=1
    freq_words = heapq.nlargest(100, wordfreq, key=wordfreq.get)
    
    freq_words_list.append(freq_words)
    temp+=1

In [84]:
temp=0
song_vectors = []
for song in songs:
    song_vec = []
    for token in freq_words_list[temp]:
        if token in song:
            song_vec.append(1)
        else:
            song_vec.append(0)
    temp+=1
    song_vectors.append(song_vec)

In [88]:
from sklearn.metrics.pairwise import cosine_similarity

temp=0
inner=temp+1
cosine_similarity_list=[]
cosine_similarity_list_row=[]

while temp<len(song_vectors)-1:
    inner=temp+1
    while inner<len(song_vectors):
        cosine_similarity_list_row.append(cosine_similarity([song_vectors[temp]],[song_vectors[inner]]))
        inner+=1
    
    cosine_similarity_list.append(cosine_similarity_list_row)
    temp+=1
    
data = cosine_similarity_list
weedf = pd.DataFrame.from_dict(data)
weedf

,0,1,2,3,4,5,6,7,8,9,...,180,181,182,183,184,185,186,187,188,189
0,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
1,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
2,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
3,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
4,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
5,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
6,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
7,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.518019126590541]],...,[[0.6886412739037006]],[[0.6130060338328881]],[[0.7551203214910351]],[[0.7565212586013159]],[[0.5372706895173354]],[[0.7930107057167913]],[[0.7321765321469982]],[[0.6049173981197841]],[[0.602763781115103]],[[0.7812429066512783]]
8,[[0.6799001036500849]],[[0.7064257574504007]],[[0.6462019508413004]],[[0.6864677122242306]],[[0.7260486951573561]],[[0.6740938478604226]],[[0.5503953220024498]],[[0.6981132075471699]],[[0.6493659782899173]],[[0.5180191265

In [46]:
len(cosine_similarity_list)

19

In [99]:
#number 4 
from sklearn.feature_extraction.text import TfidfVectorizer

lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

tfidfVec=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

def cos_sim(textlist):
    tfidf = tfidfVec.fit_transform(textlist)
    return(tfidf*tfidf.T).toarray()

data = cos_sim(songs)
tfdf = pd.DataFrame.from_dict(data)
tfdf

C:\Users\reryn\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.093670,0.073271,0.036742,0.037217,0.035011,0.061632,0.012558,0.151627,0.168328,0.003536,0.165513,0.036237,0.099731,0.173961,0.109750,0.333946,0.048816,0.094499,0.063769
1,0.093670,1.000000,0.082022,0.033488,0.060930,0.038629,0.016841,0.003269,0.055226,0.072727,0.002793,0.137360,0.016729,0.127210,0.067647,0.105729,0.084515,0.105931,0.040899,0.056675
2,0.073271,0.082022,1.000000,0.037549,0.081631,0.092000,0.162901,0.003731,0.039500,0.062996,0.008567,0.083876,0.029475,0.142664,0.054785,0.076808,0.175448,0.088098,0.086606,0.078319
3,0.036742,0.033488,0.037549,1.000000,0.013078,0.013973,0.012241,0.001928,0.018212,0.023830,0.006330,0.044280,0.399928,0.018044,0.034074,0.017022,0.057219,0.025806,0.051002,0.023256
4,0.037217,0.060930,0.081631,0.013078,1.000000,0.029228,0.039308,0.003965,0.019339,0.047584,0.003388,0.081861,0.011740,0.045774,0.070505,0.050920,0.093977,0.132024,0.065060,0.075240
5,0.035011,0.038629,0.092000,0.013973,0.029228,1.000000,0.018642,0.003027,0.017477,0.039246,0.002942,0.151342,0.008208,0.089731,0.047234,0.027823,0.094801,0.055516,0.116775,0.026296
6,0.061632,0.016841,0.162901,0.012241,0.039308,0.018642,1.000000,0.070464,0.088919,0.069121,0.075775,0.166872,0.015400,0.050753,0.266830,0.032546,0.185566,0.012937,0.099761,0.357920
7,0.012558,0.003269,0.003731,0.001928,0.003965,0.003027,0.070464,1.000000,0.058923,0.004434,0.268567,0.037637,0.000000,0.005377,0.062380,0.003873,0.027801,0.015424,0.020006,0.072051
8,0.151627,0.055226,0.039500,0.018212,0.019339,0.017477,0.088919,0.058923,1.000000,0.096343,0.012779,0.084904,0.031575,0.036158,0.053092,0.032660,0.148860,0.048277,0.052240,0.100248
9,0.168328,0.072727,0.062996,0.023830,0.047584,0.039246,0.069121,0.004434,0.096343,1.000000,0.003789,0.126536,0.045084,0.130119,0.083210,0.194900,0.200543,0.076039,0.095058,0.142439
